In [2]:
from IPython.display import Audio
import matplotlib.pyplot as plt
import tensorflow as tf
import os

from custom_metrics import f1_m, precision_m, recall_m
from utils import get_commands
from data import preprocess_dataset

TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'

<h2>Helper methods</h2>

Methods to read in audio as tensor to be able make prediction on it.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
commands = get_commands("../data/mini_speech_commands")

<h2>Load model</h2>

Load trained model from disk

In [ ]:
trained_model = tf.keras.models.load_model('../model',
                                           custom_objects={
                                               "f1_m": f1_m,
                                               "precision_m": precision_m,
                                               "recall_m": recall_m
                                           })
trained_model.summary()


<h2>Load in self spoken sample and use model make prediction</h2>

In [ ]:
sample_file = '../custom_data/commands/no/no_Joey.wav'

sample_ds = preprocess_dataset([str(sample_file)], commands)

for spectrogram, label in sample_ds.batch(1):
    prediction = trained_model(spectrogram)
    plt.bar(commands, tf.nn.softmax(prediction[0]))
    plt.title(f"Predictions {commands[label]}")
    plt.show()

Audio(sample_file, rate=16000)